# Spam Filtering Logistic Regression

- https://www.kaggle.com/uciml/sms-spam-collection-dataset  

- SMS 스팸 컬렉션은 태그가 지정된 SMS 메시지 집합입니다. 여기에는 5,574 개 메시지 중 영어로 된 SMS 메시지 한 세트가 포함되어 있으며 햄 (합법적) 또는 스팸으로 태그가 지정됩니다.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

sms = pd.read_csv("data/spam.csv", encoding="ISO-8859-1", 
                  usecols=[0, 1], skiprows=1, names=["label", "message"])

sms.head()

,label,message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


- label value 는 spam, ham 두가지  
- ham 은 0, spam 은 1 로 변환

In [2]:
sms['label'].unique()

array(['ham', 'spam'], dtype=object)

- label column을 numeric으로 변환

In [3]:
sms.label = sms.label.map({"ham": 0, "spam": 1})

In [4]:
sms.label.value_counts()

0    4825
1     747
Name: label, dtype: int64

### Train / Test split

In [5]:
X_train, X_test, y_train, y_test = train_test_split(sms.message, sms.label, test_size=0.2)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((4457,), (1115,), (4457,), (1115,))

## Text Data 전처리

### Bag of Word 문서 생성

Bag of Words란 단어들의 순서는 전혀 고려하지 않고, 단어들의 출현 빈도(frequency)에만 집중하는 텍스트 데이터의 수치화 표현 방법입니다. 텍스트 문서에 있는 단어들을 가방에다가 전부 넣습니다. 그러고나서 이 가방을 흔들어 단어들을 섞습니다. 만약, 해당 문서 내에서 특정 단어가 N번 등장했다면, 이 가방에는 그 특정 단어가 N개 있게됩니다. 또한 가방을 흔들어서 단어를 섞었기 때문에 더 이상 단어의 순서는 중요하지 않습니다.

- BoW를 만드는 과정.  
(1) 각 단어에 고유한 정수 인덱스를 부여합니다.  
(2) 각 인덱스의 위치에 단어 토큰의 등장 횟수를 기록한 벡터를 만듭니다.


- 문장을 token화 하고 각 문장에 token 이 몇 번 등장하는지 count  

- 각 token 을 feature 화하여 feature(단어) 출현 횟수에 따라 spam 여부 분류

- CountVectorizer
    - min_df : vocabulary 에 포함할 최소 발생 빈도
    - ngram_range : (1, 1) - unigram only, (1, 2) - unigram + bigram
    - max_features : top max_features 만으로 vocabulary 구성
    - token_pattern = (?u)\\b\\w\\w+\\b : unocode 영수자 2 글자 이상만 포함

### Token화된 Train document matrix  생성

In [6]:
from sklearn.feature_extraction.text import CountVectorizer

couvec = CountVectorizer()

In [8]:
X_train_tokenized = couvec.fit_transform(X_train)
X_train_tokenized

<4457x7701 sparse matrix of type '<class 'numpy.int64'>'
	with 58873 stored elements in Compressed Sparse Row format>

In [9]:
print(f"document 수: {X_train_tokenized.shape[0]}")
print(f"단어수: {X_train_tokenized.shape[1]-1}")

document 수: 4457
단어수: 7700


In [10]:
import pandas as pd
bow = pd.DataFrame(X_train_tokenized.toarray(), columns = couvec.get_feature_names_out())
bow

,00,000,000pes,008704050406,0121,01223585236,01223585334,0125698789,02,0207,...,ì¼1,ìä,ìï,û_,û_thanks,ûªve,ûï,ûïharry,ûò,ûówell
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4452,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4453,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4454,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4455,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
bow.columns[4000:4100]

Index(['lasting', 'late', 'lately', 'latelyxxx', 'later', 'latest', 'latests',
       'latr', 'laugh', 'laughing', 'laughs', 'laundry', 'laurie', 'lautech',
       'law', 'laxinorficated', 'lay', 'lays', 'lazy', 'lccltd', 'ld', 'ldew',
       'ldn', 'ldnw15h', 'le', 'lead', 'leadership', 'leading', 'leaf',
       'league', 'leanne', 'learn', 'learned', 'least', 'least5times', 'leave',
       'leaves', 'leaving', 'lect', 'lecture', 'lecturer', 'left', 'leftovers',
       'leg', 'legal', 'legs', 'leh', 'lei', 'lekdog', 'lemme', 'length',
       'lennon', 'leona', 'leonardo', 'les', 'less', 'lesser', 'lesson',
       'lessons', 'let', 'lets', 'letter', 'letters', 'level', 'lf56', 'li',
       'liao', 'lib', 'library', 'lick', 'licks', 'lido', 'lie', 'lies',
       'life', 'lifebook', 'lifeis', 'lifetime', 'lifpartnr', 'lift', 'lifted',
       'light', 'lighters', 'lightly', 'lik', 'like', 'liked', 'likely',
       'likes', 'likeyour', 'liking', 'lil', 'lily', 'limit', 'limits',
       'li

- test data 는 train data 에서 fit 한 count vectorizer 를 이용하여 transform

In [12]:
X_test_tokenized = couvec.transform(X_test)
X_test_tokenized

<1115x7701 sparse matrix of type '<class 'numpy.int64'>'
	with 13998 stored elements in Compressed Sparse Row format>

### 이진 분류기 train

In [13]:
lr_classifier = LogisticRegression(random_state=0)
lr_classifier.fit(X_train_tokenized, y_train)

LogisticRegression(random_state=0)

### predict

- predict() - 예측된 class 를 threshold 0.5 기준으로 반환
- predict_proba() - class 별 probability 를 반환

In [14]:
y_pred = lr_classifier.predict(X_test_tokenized)

print(y_pred)
print()
print("Test set 의 true counts = ", sum(y_test))
print("모델이 예측한 predicted true counts = ", sum(y_pred))
print("accuracy = {:.2f}".format(sum(y_pred == y_test) / len(y_test)))

[0 0 0 ... 0 0 0]

Test set 의 true counts =  160
모델이 예측한 predicted true counts =  143
accuracy = 0.98
